In [1]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import spacy as sp
import rich as rc
import os

In [35]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
with open("introduccion.txt", "r", encoding="utf-8") as file:
    text = file.read()

for sentence in text.split(".")[:3]:
    rc.print(sentence.strip() + ".")

Aunque su verdadero nombre era Cosme Pérez, fue conocido artísticamente como 'Juan Rana', según documenta ya la 
Genealogía  —este apodo ya se le aplicaba en 1636—.

Aunque Cotarelo aventuró que pudo haber nacido en Madrid , hoy sabemos que nació en Tudela de Duero (Valladolid), 
en cuya iglesia parroquial fue bautizado el 7 de abril de 1593.

Fue hijo de Damián Pérez e Isabel de Basto, su segunda esposa, y fue el cuarto de una familia de cuatro hermanos.

In [4]:
nlp = sp.load("es_core_news_md")
doc = nlp(text)

In [16]:
from collections import defaultdict

prompting_ners = defaultdict(set)
spacy_ners = defaultdict(set)

In [37]:
ner_extractor_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
    Extrae todas las entidades nombradas que aparezcan en el siguiente texto, sin limitarte a ningún tipo predefinido. Por cada entidad encontrada, indícala en el formato:

    TIPO_DE_NER-NER

    El tipo debe estar en mayúsculas, en ingles, y representar con claridad a qué categoría pertenece cada entidad (como PER, ORG, GPE, LOC, DATE, TIME, MONEY, PERCENT, PRODUCT, EVENT, WORK_OF_ART, LAW, LANGUAGE, etc.). Si el tipo no está claro, utiliza la mejor etiqueta posible.

    Ejemplo de formato (solo como referencia, no restrictivo):

    ### Ejemplo:

    Texto:
    "Gabriel García Márquez escribió Cien años de soledad en Colombia en 1967."

    Salida:
    PER:Gabriel García Márquez
    WORK_OF_ART:Cien años de soledad
    LOC:Colombia
    DATE:1967

    No añadas explicaciones ni comentarios. Solo devuelve una lista, una entidad por línea, en el formato indicado.

    ### Texto:
    {text}
    
    ### Salida:
    """,
)

In [38]:
llm_response = llm.invoke(ner_extractor_prompt.format(text=text)).content
llm_response

'PER:Cosme Pérez\nPER:Juan Rana\nDATE:1636\nGPE:Madrid\nLOC:Tudela de Duero\nGPE:Valladolid\nDATE:7 de abril de 1593\nPER:Damián Pérez\nPER:Isabel de Basto\nPER:Sánchez Arjona\nPER:Subirá\nPER:Bernarda Ramírez\nPER:Bernarda Manuela\nPER:Tomás Fernández\nPER:Cabredo\nPER:María de Acosta\nPER:Francisca María Pérez\nDATE:1634\nPER:Tomás Fernández\nPER:Tomás Fernández\nPER:Pedro de la Rosa\nPER:Juan Caramuel\nWORK_OF_ART:Primus calamus\nPER:Avellaneda\nWORK_OF_ART:La portería de las damas\nPER:María Teresa de Austria\nPER:Juan Rana\nPER:El Bello (o Buen) Retiro\nPER:Rey\nPER:Mariana de Austria\nPER:La portería de las damas\nPER:Pedro de la Rosa\nPER:Juan Caramuel\nWORK_OF_ART:Primus calamus\nPER:Calderón\nPER:Lope de Vega\nPER:Juan Pérez de Montalbán\nPER:Bartola, sobrina de Rana\nPER:Luis de Belmonte\nPER:Luis Quiñones de Benavente\nPER:Juan Navarro de Espinosa\nPER:Antonio de Escamilla\nPER:Roque de Figueroa\nPER:Calderón\nPER:Juan Rana\nPER:Bernarda Ramírez\nPER:Mateo Godoy\nPER:Jerónim

In [39]:
llm_response.split("\n")

['PER:Cosme Pérez',
 'PER:Juan Rana',
 'DATE:1636',
 'GPE:Madrid',
 'LOC:Tudela de Duero',
 'GPE:Valladolid',
 'DATE:7 de abril de 1593',
 'PER:Damián Pérez',
 'PER:Isabel de Basto',
 'PER:Sánchez Arjona',
 'PER:Subirá',
 'PER:Bernarda Ramírez',
 'PER:Bernarda Manuela',
 'PER:Tomás Fernández',
 'PER:Cabredo',
 'PER:María de Acosta',
 'PER:Francisca María Pérez',
 'DATE:1634',
 'PER:Tomás Fernández',
 'PER:Tomás Fernández',
 'PER:Pedro de la Rosa',
 'PER:Juan Caramuel',
 'WORK_OF_ART:Primus calamus',
 'PER:Avellaneda',
 'WORK_OF_ART:La portería de las damas',
 'PER:María Teresa de Austria',
 'PER:Juan Rana',
 'PER:El Bello (o Buen) Retiro',
 'PER:Rey',
 'PER:Mariana de Austria',
 'PER:La portería de las damas',
 'PER:Pedro de la Rosa',
 'PER:Juan Caramuel',
 'WORK_OF_ART:Primus calamus',
 'PER:Calderón',
 'PER:Lope de Vega',
 'PER:Juan Pérez de Montalbán',
 'PER:Bartola, sobrina de Rana',
 'PER:Luis de Belmonte',
 'PER:Luis Quiñones de Benavente',
 'PER:Juan Navarro de Espinosa',
 'PER:

In [40]:
for item in llm_response.split("\n"):
    try:
        ner, name = item.split(":")
    except ValueError:
        print(f"Error: {item}")
        continue
    prompting_ners[ner].add(name.strip())

In [41]:
prompting_ners

defaultdict(set,
            {'PER': {'Aguilar Priego',
              'Agustín Moreto',
              'Alonso de Olmedo',
              'Ana Coronel',
              'Andrés de Claramonte',
              'Antonio de Escamilla',
              'Antonio de Solís',
              'Avellaneda',
              'Bartola',
              'Bartola, sobrina de Rana',
              'Bergman',
              'Bernarda Manuela',
              'Bernarda Ramírez',
              'Bárbara Coronel',
              'Cabredo',
              'Calderón',
              'Cosme Pérez',
              'Cotarelo',
              'Damián Pérez',
              'Domingo Canejil',
              'El Bello (o Buen) Retiro',
              'F. Serralta',
              'F. Sáez Raposo',
              'Felipe II',
              'Francisca María Pérez',
              'I. Arellano',
              'Isabel de Basto',
              'Jerónimo Morales',
              'Juan Bautista Valenciano',
              'Juan Caramuel',
           

In [42]:
prompting_ners.keys()

dict_keys(['PER', 'DATE', 'LOC', 'WORK_OF_ART', 'GPE', 'ORG', 'SIGNATURA', 'PERSONAJE', 'EVENT'])

In [20]:
for ent in doc.ents:
    spacy_ners[ent.label_].add(ent.text)

In [21]:
spacy_ners

defaultdict(set,
            {'PER': {'Aguilar Priego',
              'Agustín Moreto',
              'Alcayde',
              'Alonso de] Olmedo',
              'Ana',
              'Ana Coronel',
              'Andrés de Claramonte',
              'Antonio Escamilla',
              'Antonio de Escamilla',
              'Antonio de Solís',
              'Antonio de] Escamilla',
              'Arellano',
              'Arte',
              'Avellaneda',
              'Bartola',
              'Bello',
              'Bergman',
              'Bernarda',
              'Bernarda Manuela',
              'Bernarda Ramírez',
              'Bárbara Coronel',
              'Calderón',
              'Calderón Fieras',
              'Calderón de la Barca',
              'Capitán Medrano',
              'Cid Campeador',
              'Cosme',
              'Cosme Pérez',
              'Cosme [Pérez',
              'Cotarelo',
              'D. D. Miller',
              'Damián Pérez',
             

In [22]:
spacy_ners.keys()

dict_keys(['PER', 'MISC', 'LOC', 'ORG'])

In [43]:
print("Entidades encontradas por el modelo de lenguaje:")
for ner_type in prompting_ners.keys():
    print(f"\t- {ner_type}: {len(prompting_ners[ner_type])} entidades")

print("\nEntidades encontradas por el modelo de Spacy:")
for ner_type in spacy_ners.keys():
    print(f"\t- {ner_type}: {len(spacy_ners[ner_type])} entidades")

Entidades encontradas por el modelo de lenguaje:
	- PER: 63 entidades
	- DATE: 37 entidades
	- LOC: 6 entidades
	- WORK_OF_ART: 40 entidades
	- GPE: 7 entidades
	- ORG: 4 entidades
	- SIGNATURA: 2 entidades
	- PERSONAJE: 3 entidades
	- EVENT: 1 entidades

Entidades encontradas por el modelo de Spacy:
	- PER: 106 entidades
	- MISC: 66 entidades
	- LOC: 43 entidades
	- ORG: 5 entidades


In [50]:
spacy_ners["LOC"]

{'Además',
 'Alcalde',
 'Aparte',
 'Biblioteca Nacional de Madrid',
 'Buen Retiro',
 'Buen) Retiro',
 'Cabredo',
 'Cantarranas',
 'Cofradía de Nuestra Señora de la Novena',
 'Cosme',
 'Cotarelo',
 'Escamilla',
 'España',
 'Espinel',
 'Grandes',
 'Grifona',
 'Guadalajara',
 'Infantes',
 'Kassel',
 'La Zarzuela',
 'Lisboa',
 'Londres',
 'Madrid',
 'New Orleans',
 'Palacio',
 'Palacio de la Zarzuela',
 'Prado',
 'Quiñones de Benavente',
 'Rana.- Verá',
 'Reichenberger',
 'Retiro',
 'Salta',
 'Salón',
 'Salón del Palacio',
 'Salón del Palacio de Buen Retiro',
 'Sebastián',
 'Su Majestad',
 'Subirá',
 'Tudela de Duero',
 'Valladolid',
 'Varey',
 'la Borja',
 'los Reyes'}

In [55]:
prompting_ners["WORK_OF_ART"]

{'A la fiesta que hizo en el Retiro a los Reyes el Príncipe de Astillano',
 'A society on Stage: Essays on Spanish Golden Drama',
 'Aguardad, supremos dioses',
 'Al cabo de los bailes mil',
 'Bien vengas mal',
 'Del horror a la risa: los géneros teatrales clásicos',
 'El caballero novel',
 'El desdén vengado',
 'El doctor Juan Rana',
 'El guardainfante',
 'El mago',
 'El mundo',
 'El mundo al revés',
 'El niño caballero',
 'El poeta de bailes',
 'El poeta de bailes y el letrado',
 'El remediador',
 'El retrato de Juan Rana',
 'El segundo Séneca de España',
 'El soldado',
 'El triunfo de Juan Rana',
 'Entremeses',
 'Fieras afemina amor',
 'Genealogía',
 'Juan Rana, a Gay Golden Age Gracioso',
 'La Zarzuela',
 'La infelice Dorotea',
 'La loa de Juan Rana',
 'La nueva victoria de don Gonzalo de Córdoba',
 'La portería de las damas',
 'Las fiestas bacanales',
 'Lo que ha de ser',
 'Los muertos vivos',
 'Los volatines',
 'Pipote',
 'Pipote en nombre de Juan Rana',
 'Primus calamus',
 'Salta